# Combinando Datasets: Merge y Join

Una característica esencial que ofrece Pandas son sus operaciones de unión y fusión en memoria de alto rendimiento.
Si alguna vez ha trabajado con bases de datos, debería estar familiarizado con este tipo de interacción de datos.
La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo esto puede funcionar en la práctica.

Por conveniencia, comenzaremos redefiniendo la funcionalidad ``display()`` de la sección anterior:

In [1]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Álgebra relacional

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para manipular datos relacionales y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.
La fortaleza del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los componentes básicos de operaciones más complicadas en cualquier conjunto de datos.
Con este léxico de operaciones fundamentales implementado eficientemente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.
Como veremos, estos le permiten vincular de manera eficiente datos de diferentes fuentes.

## Categorías de uniones

La función ``pd.merge()`` implementa varios tipos de uniones: las uniones *uno a uno*, *muchos a uno* y *muchos a muchos*.
Se accede a los tres tipos de combinaciones mediante una llamada idéntica a la interfaz ``pd.merge()``; El tipo de unión realizada depende de la forma de los datos de entrada.
Aquí mostraremos ejemplos simples de los tres tipos de fusiones y analizaremos las opciones detalladas más adelante.

### Unión uno a uno

Quizás el tipo más simple de expresión de fusión es la unión uno a uno, que en muchos aspectos es muy similar a la concatenación de columnas que se ve en Concat & Append.
Como ejemplo concreto, considere los siguientes dos ``DataFrames`` que contienen información sobre varios empleados de una empresa:

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un único ``DataFrame``, podemos usar la función ``pd.merge()``:

In [3]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [4]:
type(df3)

pandas.core.frame.DataFrame

La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna de "empleado" y se une automáticamente usando esta columna como clave.
El resultado de la fusión es un nuevo ``DataFrame`` que combina la información de las dos entradas.
Tenga en cuenta que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna "empleado" difiere entre ``df1`` y ``df2``, y ``pd.merge()`` la función tiene en cuenta esto correctamente.
Además, tenga en cuenta que la fusión en general descarta el índice, excepto en el caso especial de fusiones por índice (consulte las palabras clave ``left_index`` y ``right_index``, que se analizan momentáneamente).

### Unión muchos a uno

Las combinaciones de muchos a uno son combinaciones en las que una de las dos columnas clave contiene entradas duplicadas.
Para el caso de muchos a uno, el ``DataFrame`` resultante conservará esas entradas duplicadas según corresponda.
Considere el siguiente ejemplo de una unión de muchos a uno:

In [5]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR',],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en una o más ubicaciones según lo requieran las entradas.

### Unión muchos a muchos

Las uniones de muchos a muchos son un poco confusas conceptualmente, pero aun así están bien definidas.
Si la columna clave en la matriz izquierda y derecha contiene duplicados, entonces el resultado es una combinación de muchos a muchos.
Quizás esto quede más claro con un ejemplo concreto.
Considere lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.
Al realizar una unión de muchos a muchos, podemos recuperar las habilidades asociadas a cualquier persona individual:

In [6]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de uniones se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funciones.
Pero en la práctica, los conjuntos de datos rara vez son tan limpios como con el que estamos trabajando aquí.
En la siguiente sección consideraremos algunas de las opciones proporcionadas por ``pd.merge()`` que le permiten ajustar cómo funcionan las operaciones de unión.

## Especificación de la clave de combinación

Ya hemos visto el comportamiento predeterminado de ``pd.merge()``: busca uno o más nombres de columnas coincidentes entre las dos entradas y lo usa como clave.
Sin embargo, a menudo los nombres de las columnas no coinciden tan bien y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

### La palabra clave ``on``

Lo más simple es que puedes especificar explícitamente el nombre de la columna clave usando la palabra clave ``on``, que toma un nombre de columna o una lista de nombres de columnas:

In [7]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción sólo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

### Las palabras clave ``left_on`` y ``right_on``

En ocasiones, es posible que desee fusionar dos conjuntos de datos con nombres de columnas diferentes; por ejemplo, es posible que tengamos un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado".
En este caso, podemos usar las palabras clave ``left_on`` y ``right_on`` para especificar los nombres de las dos columnas:

In [8]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')
# Solo se puede de 2 en 2

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, usando el método ``drop()`` de ``DataFrames``:

In [9]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Las palabras clave ``left_index`` y ``right_index``

A veces, en lugar de fusionar en una columna, le gustaría fusionar en un índice.
Por ejemplo, sus datos podrían verse así:

In [10]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Puedes usar el índice como clave para fusionar especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [11]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Para mayor comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza una fusión que por defecto se une en índices:

In [12]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Si desea mezclar índices y columnas, puede combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [13]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')[['name','group','salary']]")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')[['name','group','salary']]
   name        group  salary
0   Bob   Accounting   70000
1  Jake  Engineering   80000
2  Lisa  Engineering  120000
3   Sue           HR   90000

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; La interfaz para este comportamiento es muy intuitiva.
Para obtener más información sobre esto, consulte la ["Merge, Join, and Concatenate" section](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

## Especificación de aritmética de conjuntos para uniones

En todos los ejemplos anteriores hemos pasado por alto una consideración importante al realizar una unión: el tipo de aritmética de conjuntos utilizada en la unión.
Esto aparece cuando aparece un valor en una columna clave pero no en la otra. Considere este ejemplo:

In [14]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos fusionado dos conjuntos de datos que tienen una sola entrada de "nombre" en común: María.
De forma predeterminada, el resultado contiene la *intersección* de los dos conjuntos de entradas; esto es lo que se conoce como *unión interna*.
Podemos especificar esto explícitamente usando la palabra clave ``how``, que por defecto es ``"inner"``:

In [15]:
pd.merge(df6, df7, how='inner') #inner es como interseccion

,name,food,drink
0,Mary,bread,wine


Otras opciones para la palabra clave ``how`` son ``'outer'``, ``'left'`` y ``'right'``.
Una *unión externa* devuelve una unión sobre la unión de las columnas de entrada y completa todos los valores faltantes con NA:

In [16]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')") #outer es como union

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

Las acciones *left join* and *right join* devuelven los registros del conjunto de la izquierda y de la derecha, respectivamente.
Por ejemplo:

In [17]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")
# le da preferencia al de la izquierda, es decir
# se mantiene al hacer merge

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

Las filas de salida ahora corresponden a las entradas en la entrada izquierda. Usando
``how='right'`` funciona de manera similar.

Todas estas opciones se pueden aplicar directamente a cualquiera de los tipos de unión anteriores.

In [18]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")
# preferencia al de la derecha

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='right')
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

## Nombres de las columnas superpuestas: ``suffixes`` Keyword

Finalmente, puede terminar en un caso en el que sus dos ``DataFrames`` de entrada tengan nombres de columnas en conflicto.
Considere este ejemplo:

In [19]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')
# con on='' especifico que haga el merge por 'name'

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Debido a que la salida tendría dos nombres de columnas en conflicto, la función de combinación agrega automáticamente un sufijo ``_x`` o ``_y`` para que las columnas de salida sean únicas.
Si estos valores predeterminados no son apropiados, es posible especificar un sufijo personalizado usando la palabra clave ``suffixes``:

In [20]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Estos sufijos funcionan en cualquiera de los patrones de unión posibles y también funcionan si hay varias columnas superpuestas.

Para obtener más información sobre estos patrones, consulte Aggregation and Grouping donde profundizamos un poco más en el álgebra relacional.
Vea también el [Pandas "Merge, Join and Concatenate" documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html) para una mayor discusión sobre estos temas.

## Ejemplo: datos de los estados de EE. UU.

Las operaciones de fusión y unión surgen con mayor frecuencia cuando se combinan datos de diferentes fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE. UU. y sus poblaciones.
Los archivos de datos se pueden encontrar en http://github.com/jakevdp/data-USstates/:

In [21]:
# # Los siguientes son comandos de shell para descargar los datos (ya descargados)
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Echemos un vistazo a los tres conjuntos de datos, usando la función ``read_csv()`` de Pandas:

In [22]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Dada esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE. UU. según su densidad de población en 2010.
Claramente tenemos los datos aquí para encontrar este resultado, pero tendremos que combinar los conjuntos de datos para encontrar el resultado.

Comenzaremos con una fusión de muchos a uno que nos dará el nombre completo del estado dentro del ``DataFrame`` poblacion.
Queremos fusionarnos en función de la columna ``state/región`` de ``pop`` y la columna ``abbreviation`` de ``abbrevs``.
Usaremos ``how='outer'`` para asegurarnos de que no se desperdicie ningún dato debido a etiquetas que no coinciden.

In [23]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Verifiquemos nuevamente si hubo discrepancias aquí, lo cual podemos hacer buscando filas con valores nulos:

In [24]:
merged.isnull().any() #any funciona por series (columna a columna)

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Parte de la información de la ``population`` es nula; ¡Averigüemos cuáles son!

In [25]:
merged.isnull().sum() #me devuelve cuantos NaN hay en cada columna

state/region     0
ages             0
year             0
population      20
state           96
dtype: int64

In [26]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; Es probable que esto se deba a que estos datos no están disponibles en la fuente original.

Más importante aún, vemos también que algunas de las nuevas entradas de ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [27]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos inferir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (EE.UU.), mientras que estas entradas no aparecen en la clave de abreviatura estatal.
Podemos solucionarlos rápidamente completando las entradas apropiadas:

In [28]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No más valores nulos en la columna ``state``: ¡ya estamos listos!

Ahora podemos fusionar el resultado con los datos del área usando un procedimiento similar.
Al examinar nuestros resultados, querremos unirnos en la columna ``state`` en ambos:

In [29]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Nuevamente, revisemos los valores nulos para ver si hubo alguna discrepancia:

In [30]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [31]:
final.isnull().sum()

state/region      0
ages              0
year              0
population       20
state             0
area (sq. mi)    48
dtype: int64

Hay valores nulos en la columna ``área``; Podemos echar un vistazo para ver qué regiones se ignoraron aquí:

In [32]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Vemos que nuestras ``areas`` ``DataFrame`` no contiene el área de Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (usando la suma de todas las áreas estatales, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [33]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta de interés, seleccionemos primero la parte de los datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()``

In [34]:
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

In [35]:
data2010 = final.query("year == 2010 & ages == 'total'") #filtrado mas sencillo
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Ahora calculemos la densidad de población y mostremosla en orden.
Comenzaremos reindexando nuestros datos sobre el estado y luego calcularemos el resultado:

In [36]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [37]:
density

state
Alabama                   91.287603
Alaska                     1.087509
Arizona                   56.214497
Arkansas                  54.948667
California               228.051342
Colorado                  48.493718
Connecticut              645.600649
Delaware                 460.445752
District of Columbia    8898.897059
Florida                  286.597129
Georgia                  163.409902
Hawaii                   124.746707
Idaho                     18.794338
Illinois                 221.687472
Indiana                  178.197831
Iowa                      54.202751
Kansas                    34.745266
Kentucky                 107.586994
Louisiana                 87.676099
Maine                     37.509990
Maryland                 466.445797
Massachusetts            621.815538
Michigan                 102.015794
Minnesota                 61.078373
Mississippi               61.321530
Missouri                  86.015622
Montana                    6.736171
Nebraska              

In [38]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

El resultado es una clasificación de los estados de EE. UU. más Washington, DC y Puerto Rico en orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington, DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos consultar el final de la lista:

In [39]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Vemos que el estado menos denso, con diferencia, es Alaska, con un promedio de poco más de un residente por milla cuadrada.

Este tipo de combinación de datos desordenada es una tarea común cuando se intenta responder preguntas utilizando fuentes de datos del mundo real.
Espero que este ejemplo le haya dado una idea de las formas en que puede combinar las herramientas que hemos cubierto para obtener información a partir de sus datos.